In [87]:
inputExample = open('day19-example.txt', 'r').read().split('\n\n')
inputExample2 = open('day19-example2.txt', 'r').read().split('\n\n')
inputReal = open('day19.txt', 'r').read().split('\n\n')

import re
from collections import deque

In [2]:
def parse(input):
    rules = {}
    for r in map(lambda x: x.split(': '), input[0].split('\n')):
        rules.update({int(r[0]) : r[1]})
        
    testCases = input[1].split('\n')
    
    return (rules, testCases)

In [3]:
def build(rules, rule):
    subRules = rules[rule].split(' ')

    regex = "("

    for sr in subRules:
        if sr[0] == '"':
            return sr[1]
        elif sr[0] == '|':
            regex += "|"
        else:
            regex += build(rules, int(sr))

    regex += ")"
    return regex

In [4]:
def part1(input):
    (rules, testCases) = parse(input)

    expression = ("^" + build(rules, 0) + "$")

    count = 0
    for m in testCases:
        if re.search(expression, m):
            count += 1

    print(count)

In [5]:
part1(inputExample)

2


In [6]:
part1(inputReal)

226


## Part 2

This feels like a substantial step up from part 1. If you've done this like me and recognised the grammar described by the rules is a [Chomsky Type-3 grammar](https://en.wikipedia.org/wiki/Chomsky_hierarchy) (ie, a regex), and used built-in regex tooling to solve the issue.

The addition of loops isn't fatal to the use of regular expressions for this the replacement rule 8 (`8: 42 | 42 8`) is representable in regex as `(<RULE42>)+` - all it's doing is repeating rule 42 one or more times.

However, the new rule 11 (`11: 42 31 | 42 11 31`) **is** fatal to the use of regexes. This one rule transforms the grammar as a whole from a Type-3 to a Type-2 (context-free) grammar. This is because there has to be a matching number of 42s and 31s.

Luck is, however, on our side. I'm not particularly interested in writing a full parser for *all* the rules - that sounds like far too much effort for an AoC problem. However, the new rules 8 and 11 are called from exactly one other rule - rule 0. Additionally, both rules 8 and 11 only call (themselves, or) rules 42 and 31, and everything below those rules can still be considered as a Type-3 grammar. Let's see if we can build a parser, where

Through some manual shuffling of the example2 rules, we can determine that the valid examples are all fixed-length of size 15. (Right column is the length of string the rule matches)

```
0: 8 11            15
1: "a"             1
2: 1 24 | 14 4     3
3: 5 14 | 16 1     3
4: 1 1             2
5: 1 14 | 15 1     2
6: 14 14 | 1 14    2
7: 14 5 | 1 21     3
8: 42              5
9: 14 27 | 1 26    4
10: 23 14 | 28 1   4
11: 42 31          10
12: 24 14 | 19 1   3
13: 14 3 | 1 12    4
14: "b"            1
15: 1 | 14         1
16: 15 1 | 14 14   2
17: 14 2 | 1 7     4
18: 15 15          2
19: 14 1 | 14 14   2
20: 14 14 | 1 15   2
21: 14 1 | 1 14    2
22: 14 14          2
23: 25 1 | 22 14   3
24: 14 1           2
25: 1 1 | 1 14     2
26: 14 22 | 1 20   3
27: 1 6 | 14 18    3
28: 16 1           3
31: 14 17 | 1 13   5
42: 9 14 | 10 1    5
```

In [7]:
def scanRules(input):
    ruleTotals = {}
    (unprocessedRules, _) = parse(input)
    changed = True
    while changed:
        changed = False
        for r in unprocessedRules:
            subRules = unprocessedRules[r].split(' ')
            length = 0
            secondLength = None
            issueEncountered = False
            for sr in subRules:
                if sr[0] == '"':
                    length = 1
                    break
                elif sr == '|':
                    secondLength = length
                    length = 0
                else:
                    if int(sr) in ruleTotals:
                        length += ruleTotals[int(sr)]
                    else:
                        issueEncountered = True
                        break
            if issueEncountered:
                continue

            if secondLength is not None and length != secondLength:
                raise Exception("Differing lengths detected!")

            unprocessedRules.pop(r)
            ruleTotals.update({r: length})
            changed = True
            break

    return ruleTotals

print("Example:")
ruleTotalsExample = scanRules(inputExample2)
print("rule 0 length:", ruleTotalsExample[0])
print()
print("Real input:")
ruleTotalsReal = scanRules(inputReal)
print("rule 0 length:", ruleTotalsReal[0])

Example:
rule 0 length: 15

Real input:
rule 0 length: 24


From this, we can see my input *also* has a fixed length (of 24) for rule 0.

We need to replace rules 8 and 11 with the following:
```
8: 42 | 42 8
11: 42 31 | 42 11 31
```

To make this easier, we'll just check the values for rules 42 and 31:

In [33]:
print("rule 31 length:", ruleTotalsExample[31])
print("rule 42 length:", ruleTotalsExample[42])
print()
print("rule 31 length:", ruleTotalsReal[31])
print("rule 42 length:", ruleTotalsReal[42])

rule 31 length: 5
rule 42 length: 5

rule 31 length: 8
rule 42 length: 8


Let's just use the regexes for these rules to make the overall parser easier, and treat 31 and 42 as terminal symbols for the grammar.

In [22]:
(rules, testCases) = parse(inputExample2)

In [23]:
print(build(rules, 42))

((b(a(bb|ab)|b((a|b)(a|b)))|a(b(bb)|a(bb|a(a|b))))b|(((aa|ab)a|(bb)b)b|(((a|b)a|bb)a)a)a)


In [19]:
print(build(rules, 31))

(b(b(a(ba)|b(aa))|a(b(ab|(a|b)a)|a(ba|ab)))|a(b((ab|(a|b)a)b|((a|b)a|bb)a)|a((ba)b|(ba|bb)a)))


#### The plan

1. Tokenise the input into a token stream (using tokens 31, 42, and -1)
2. Build real code to parse rules 8 and 11 based on the token stream
3. ???
4. Profit!

New ruleset that I care about:
```
8: 42 | 42 8
11: 42 31 | 42 11 31
0: 8 11
```

This basically means:
* At least 1x 42, but possibly many; followed by
* At least 1x 42/31 pair, but possibly many.

So, we need to look for the following cases:
* All 42s at the start
* All 31s at the end
* At least 1 more 42 than 31.

In [73]:
def matchToken(case, expression, expressionLength):
    if re.search(expression, case[0:expressionLength]):
        return True
    else:
        return False

In [89]:
def part2(input):
    (rules, testCases) = parse(input)
    ruleLengths = scanRules(input)
        
    rules.update({8:"42 | 42 8"})
    rules.update({11:"42 31 | 42 11 31"})

    ruleExpressions = {}
    
    print("Using rules: ")
    for r in [31, 42]:
        ruleExpressions[r] = build(rules,r)
        print(r, ruleLengths[r], ruleExpressions[r])

    print()
    
    totalValid = 0
    
    for case in testCases:
        tokenStream = []
        position = 0
        maxPosition = len(case)
        
        while position + 1 <= len(case):
            if matchToken(case[position:], ruleExpressions[42], ruleLengths[42]):
                position += ruleLengths[42]
                tokenStream.append(42)
            elif matchToken(case[position:], ruleExpressions[31], ruleLengths[31]):
                position += ruleLengths[31]
                tokenStream.append(31)
            else:
                tokenStream.append(case[position:position+1])
                position += 1
                
        count42 = 0
        count31 = 0
        
        valid = True
        
        for token in tokenStream:
            if token == 42:
                count42 += 1
                
                if count31 > 0:
                    print("", "** out of order tokens")
                    valid = False
            if token == 31:
                count31 += 1
        
        if count42 <= count31:
            print("", "** count mismatch", count42, count31)
            valid = False
        
        if count31 == 0:
            print("" "** 31 missing")
            valid = False
        
        if valid:
            totalValid += 1
        
        print(valid, case, tokenStream)
    print(totalValid)

In [90]:
part2(inputExample2)

Using rules: 
31 5 (b(b(a(ba)|b(aa))|a(b(ab|(a|b)a)|a(ba|ab)))|a(b((ab|(a|b)a)b|((a|b)a|bb)a)|a((ba)b|(ba|bb)a)))
42 5 ((b(a(bb|ab)|b((a|b)(a|b)))|a(b(bb)|a(bb|a(a|b))))b|(((aa|ab)a|(bb)b)b|(((a|b)a|bb)a)a)a)

 ** out of order tokens
 ** out of order tokens
 ** out of order tokens
 ** out of order tokens
False abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa [42, 42, 42, 31, 42, 31, 42, 42, 42]
True bbabbbbaabaabba [42, 42, 31]
True babbbbaabbbbbabbbbbbaabaaabaaa [42, 42, 42, 42, 31, 31]
True aaabbbbbbaaaabaababaabababbabaaabbababababaaa [42, 42, 42, 42, 42, 31, 31, 31, 31]
True bbbbbbbaaaabbbbaaabbabaaa [42, 42, 42, 42, 31]
True bbbababbbbaaaaaaaabbababaaababaabab [42, 42, 42, 42, 42, 31, 31]
True ababaaaaaabaaab [42, 42, 31]
True ababaaaaabbbaba [42, 42, 31]
True baabbaaaabbaaaababbaababb [42, 42, 42, 31, 31]
True abbbbabbbbaaaababbbbbbaaaababb [42, 42, 42, 42, 42, 31]
True aaaaabbaabaaaaababaa [42, 42, 42, 31]
** 31 missing
False aaaabbaaaabbaaa [42, 42, 42]
True aaaabbaabbaaaaaaabbbab

In [91]:
part2(inputReal)

Using rules: 
31 8 (a(((a(b(b(bb|aa)|a(aa))|a(a((b|a)(b|a))|b(ba|aa)))|b(b((ba|ab)a|(bb|ab)b)|a((bb)a|(ba|aa)b)))b|(((b(ba)|a((b|a)(b|a)))a|((ab)a|(bb)b)b)a|(((ab)b|(aa|(b|a)b)a)a|((ba|a(b|a))a|(bb|aa)b)b)b)a)a|(((b(b(ab|b(b|a))|a((b|a)(b|a)))|a(a((b|a)(b|a))|b(ba|bb)))b|((b(ba|a(b|a))|a(aa))b|(a(ba)|b(ab|aa))a)a)b|((b(b(aa|(b|a)b)|a(ab))|a(b(ab|b(b|a))|a(aa)))a|((b(aa)|a(ba|bb))b|((aa|(b|a)b)a|(ba|bb)b)a)b)a)b)|b(a((((b(bb)|a(bb|ab))a|(b(ba)|a(ba|aa))b)b|(((bb)b|(bb|aa)a)b|((ab)a|(ba)b)a)a)a|((a(a(ba|a(b|a))|b(ba))|b((bb)a|(ba|aa)b))a|((a((b|a)(b|a))|b(ab))(b|a))b)b)|b((((b(ab|b(b|a))|a(aa))b|(a((b|a)(b|a))|b(ba|bb))a)b|(a((ba|aa)a|(bb|ab)b)|b(a(ba|ab)|b(aa)))a)a|(((a((b|a)(b|a))|b(ab))b|(b(ba|ab)|a(ba|bb))a)a|((b(bb)|a(bb|ab))a|((aa)b|(ab|aa)a)b)b)b)))
42 8 (a((((b((ba|a(b|a))a|(bb|aa)b)|a(b(bb|ab)))a|((b(ab|aa)|a(ba|ab))b|((bb|ab)a|(aa|(b|a)b)b)a)b)b|((b(b(ba|ab)|a((b|a)(b|a)))|a(b(bb)|a(bb|ab)))a|(b((aa|(b|a)b)a|(ba|ab)b)|a((ab)a|(bb)b))b)a)a|(a(a((b((b|a)(b|a))|a(ba|bb))a|(b(ba|aa